In [333]:
from LinkedBinaryTree import * 
import random
from game_tree import *


def build_set_tree_with_weights(tree):
    construct_tree(tree)

    visited = set()

    def assign_weights(node, win_on_receive_weight, momentum):
        if node is None or node in visited:
            return

        visited.add(node)

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            node.right_weight = 0.5  # Default for winning
            node.left_weight = 0.5 # Default for losing

            if "40" in node.data and "0" not in node.data:
                node.right_weight += win_on_receive_weight

            if "15" in node.data or "30" in node.data:  # Example condition for momentum
                node.right_weight += momentum

            # Adjust weights if momentum is against the player
            if "0-" in node.data or "15-" in node.data or "30-0" in node.data:
                node.right_weight -= momentum
                node.left_weight += momentum

            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight

        assign_weights(node.left, win_on_receive_weight, momentum)
        assign_weights(node.right, win_on_receive_weight, momentum)

    assign_weights(tree.root, 0.1, 0.05)

def simulate_set(tree):
    player_games = 0
    opponent_games = 0

    while True:
        current_node = tree.root

        while current_node:
            if current_node.data == "WIN":
                player_games += 1
                break
            elif current_node.data == "LOSE":
                opponent_games += 1
                break

            random_value = random.random()
            if random_value < current_node.right_weight and current_node.right:
                current_node = current_node.right
            elif current_node.left:
                current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)

result, player_games, opponent_games = simulate_set(binary_tree)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


Opponent wins the set
Player Games: 4, Opponent Games: 6


In [334]:
# def predict_set_winner_simulation(tree, match_data):
#     player_games = int(match_data["p1_games"])
#     opponent_games = int(match_data["p2_games"])
#     player_score = int(match_data["p1_score"])
#     opponent_score = int(match_data["p2_score"])

#     while True:
#         current_node = tree.root
#         while current_node:
#             random_value = random.random()
#             if current_node.data == "WIN":
#                 player_score += 1
#                 if player_score == 4:  # Win game
#                     player_games += 1
#                     player_score = 0
#                     opponent_score = 0
#                 break
#             elif current_node.data == "LOSE":
#                 opponent_score += 1
#                 if opponent_score == 4:  # Lose game
#                     opponent_games += 1
#                     player_score = 0
#                     opponent_score = 0
#                 break

#             if random_value < current_node.right_weight and current_node.right:
#                 current_node = current_node.right
#             elif current_node.left:
#                 current_node = current_node.left

#         if player_games >= 6 and player_games - opponent_games >= 2:
#             return "Player wins the set", player_games, opponent_games
#         if opponent_games >= 6 and opponent_games - player_games >= 2:
#             return "Opponent wins the set", player_games, opponent_games

# binary_tree = LinkedBinaryTree()
# build_set_tree_with_weights(binary_tree)


In [335]:
def build_set_tree_with_weights(tree, match_data):
    construct_tree(tree)

    visited = set()

    def assign_weights(node, win_on_receive_weight, momentum):
        if node is None or node in visited:
            return

        visited.add(node)

        # Determine who has momentum and advantage
        server = int(match_data["server"])
        point_victor = int(match_data["point_victor"])
        p1_break_pt = int(match_data["p1_break_pt_won"])
        p2_break_pt = int(match_data["p2_break_pt_won"])

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            node.right_weight = 0.5
            node.left_weight = 0.5

            if server == 1 and p1_break_pt:
                node.right_weight += momentum
            elif server == 2 and p2_break_pt:
                node.left_weight += momentum

            if point_victor == 1:  # Player 1 has momentum
                node.right_weight += win_on_receive_weight
            elif point_victor == 2:  # Player 2 has momentum
                node.left_weight += win_on_receive_weight

            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight
            else:
                node.right_weight = 0.5
                node.left_weight = 0.5


        assign_weights(node.left, win_on_receive_weight, momentum)
        assign_weights(node.right, win_on_receive_weight, momentum)

    assign_weights(tree.root, 0.1, 0.05)

def find_start_node(tree, current_score):
    if tree.root is None:
        raise ValueError("The tree root is None. Ensure the tree is constructed properly.")

    stack = [tree.root]
    visited = set()
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        if node.data == current_score:
            return node
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    raise ValueError(f"Score {current_score} not found in tree. Verify tree construction and current_score format.")


def predict_set_winner_simulation(tree, match_data):
    player_games = int(match_data["p1_games"])
    opponent_games = int(match_data["p2_games"])
    player_score = match_data["p1_score"]
    opponent_score = match_data["p2_score"]
    current_score = f"{player_score}-{opponent_score}"

    current_node = find_start_node(tree, current_score)
    if current_node is None:
        raise ValueError(f"Score {current_score} not found in tree")

    while True:

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

        random_value = random.random()
        if current_node.data == "WIN":
            player_games += 1
            current_node = tree.root
            continue
        elif current_node.data == "LOSE":
            opponent_games += 1
            current_node = tree.root
            continue

        if random_value < current_node.right_weight and current_node.right:
            current_node = current_node.right
        elif current_node.left:
            current_node = current_node.left
        else:
            print("No valid child node. Ending simulation.")
            break

binary_tree = LinkedBinaryTree()

In [336]:
import pandas as pd
from collections import Counter


# Load match data
match_data_df = pd.read_csv("m1.csv")

# Convert the first row of the DataFrame to a dictionary
match_data = match_data_df.iloc[-1].to_dict()

print(match_data)
# Initialize the binary tree
binary_tree = LinkedBinaryTree()

# Build the tree with weights using the match data
build_set_tree_with_weights(binary_tree, match_data)

results = []
for _ in range(100):
    result, player_games, opponent_games = predict_set_winner_simulation(binary_tree, match_data)
    results.append((result, player_games, opponent_games))

# Count the number of times Player 1 wins
player1_wins = sum(1 for result, _, _ in results if result == "Player wins the set")

# Get distribution of scores
score_distribution = Counter((player_games, opponent_games) for _, player_games, opponent_games in results)

print(f"Player 1 Wins: {player1_wins}")
print("Score Distribution:")
for score, count in score_distribution.items():
    print(f"{score}: {count}")


{'match_id': '2023-wimbledon-1301', 'player1': 'Carlos Alcaraz', 'player2': 'Nicolas Jarry', 'elapsed_time': '00:37:18', 'set_no': 1, 'game_no': 9, 'point_no': 52, 'p1_sets': 0, 'p2_sets': 0, 'p1_games': 5, 'p2_games': 3, 'p1_score': '0', 'p2_score': '0', 'server': 1, 'serve_no': 1, 'point_victor': 1, 'p1_points_won': 28, 'p2_points_won': 24, 'game_victor': 0, 'set_victor': 0, 'p1_ace': 0, 'p2_ace': 0, 'p1_winner': 0, 'p2_winner': 0, 'winner_shot_type': '0', 'p1_double_fault': 0, 'p2_double_fault': 0, 'p1_unf_err': 0, 'p2_unf_err': 0, 'p1_net_pt': 0, 'p2_net_pt': 0, 'p1_net_pt_won': 0, 'p2_net_pt_won': 0, 'p1_break_pt': 0, 'p2_break_pt': 0, 'p1_break_pt_won': 0, 'p2_break_pt_won': 0, 'p1_break_pt_missed': 0, 'p2_break_pt_missed': 0, 'p1_distance_run': 3.341, 'p2_distance_run': 4.725, 'rally_count': 1, 'speed_mph': 115.0, 'serve_width': 'BW', 'serve_depth': 'NCTL', 'return_depth': 'D'}
Player 1 Wins: 94
Score Distribution:
(6, 3): 62
(6, 4): 23
(7, 5): 4
(5, 7): 5
(9, 7): 1
(10, 8): 1
(